In [1]:
import os
import yaml
import plotly.graph_objects as go
import plotly.io as pio

In [6]:
def render_latex_plotly(latex_str: str, output_base: str,
                        width: int = 800, height: int = 200, font_size: int = 24):
    """
    Renders a LaTeX string using Plotly annotations and exports PNG + SVG via Kaleido.
    """
    fig = go.Figure()
    fig.add_annotation(
        text=f"$$\n{latex_str}\n$$",
        xref="paper", yref="paper",
        x=0.5, y=0.5,
        showarrow=False,
        font=dict(size=font_size),
    )
    # fig.update_layout(
    #     width=width, height=height,
    #     margin=dict(l=0, r=0, t=0, b=0),
    #     xaxis=dict(visible=False), yaxis=dict(visible=False),
    # )
    fig.show()
    # os.makedirs(os.path.dirname(output_base), exist_ok=True)
    # fig.write_image(f"{output_base}.png", format="png", engine="kaleido")
    # fig.write_image(f"{output_base}.svg", format="svg", engine="kaleido")

In [7]:
yaml_path = "minimal_fork_of_putnambench_with_clear_answers.yaml"

In [8]:
# 1) load YAML
with open(yaml_path, "r") as f:
    problems = yaml.safe_load(f)

# 2) process each entry
# for item in problems:
#     name = item["problem_name"]
#     stmt = item["informal_statement"]
#     # if you only want the math between \[...\], you can extract via regex:
#     # import re
#     # exprs = re.findall(r"\\\[(.*?)\\\]", stmt, re.DOTALL)
#     # latex_to_render = "\n\n".join(exprs) if exprs else stmt

#     # here we render the full statement (text + math)
#     output_base = os.path.join(out_dir, name + "_stmt")
#     render_latex_plotly(stmt, output_base)

In [9]:
name = problems[0]["problem_name"]
stmt = problems[0]["informal_statement"]
output_base = os.path.join("putnam_problems_images", name + "_stmt")
render_latex_plotly(stmt, output_base)


In [1]:
import asyncio
import logging
import os
import uuid
from pathlib import Path
from typing import Any, Optional

# import click
import pandas as pd
import yaml

# from chainscope.api_utils.deepseek_utils import (
#     DeepSeekBatchProcessor,
#     DeepSeekRateLimiter,
# )
# from chainscope.api_utils.open_router_utils import ORBatchProcessor, ORRateLimiter
from chainscope.typing import (
    CotResponses,
    DefaultSamplingParams,
    MathDatasetParams,
    MathQsDataset,
    MathQuestion,
    MathResponse,
)

import base64


In [2]:
def load_putnam_results_as_df(yaml_path: Path) -> pd.DataFrame:
    """Load Putnam results from YAML into a pandas DataFrame."""
    with open(yaml_path) as f:
        data = yaml.safe_load(f)
    return pd.DataFrame(data)

In [4]:
data_df = load_putnam_results_as_df("minimal_fork_of_putnambench_with_clear_answers.yaml")

In [5]:
data_df

,problem_name,informal_statement,informal_solution
0,putnam_1962_a2,Find every real-valued function $f$ whose doma...,Show that \[ f(x) = \frac{a}{(1 - cx)^2} \begi...
1,putnam_1962_a5,Evaluate in closed form \[ \sum_{k=1}^n {n \ch...,Show that the expression equals $n(n+1)2^{n-2}$.
2,putnam_1963_a3,"Find an integral formula (i.e., a function $z$...",Show that the solution is $$y(x) = \int_{1}^{x...
3,putnam_1963_b1,For what integer $a$ does $x^2-x+a$ divide $x^...,Show that $a=2$.
4,putnam_1963_b3,Find every twice-differentiable real-valued fu...,Show that the solution is the sets of function...
...,...,...,...
210,putnam_2023_b1,"Consider an $m$-by-$n$ grid of unit squares, i...",Show that the number of such configurations is...
211,putnam_2023_b3,"A sequence $y_1, y_2, \ldots, y_k$ of real num...",Show that the expected value is \frac{2n + 2}{3}.
212,putnam_2023_b4,For a nonnegative integer $n$ and a strictly i...,Show that the minimum value of $T$ is $29$.
213,putnam_2023_b5,Determine which positive integers $n$ have the...,Show that the desired property holds if and on...


In [21]:
filtered = []

In [22]:
for index, row in data_df.iterrows():
    problem_name = row["problem_name"]
    image_path = f"putnam_problems_images/{problem_name}_stmt.png"
    if os.path.exists(image_path):
        filtered.append({
            "problem_name": row["problem_name"],
            "informal_statement": row["informal_statement"],
            "informal_solution": row["informal_solution"]
        })
    else:
        continue


In [23]:
with open("putnam_problems.yaml", "w", encoding="utf-8") as f:
    yaml.safe_dump(filtered, f, allow_unicode=True, default_flow_style=False)

In [31]:
import yaml

def filter_putnam_problems(putnam_yaml_path, claude_yaml_paths, output_path):
    # 1. Load all problem names from the large file (using grep output for efficiency)
    claude_names = set()
    for claude_yaml_path in claude_yaml_paths:
        with open(claude_yaml_path, encoding='utf-8') as f:
            for line in f:
                if line.strip().startswith('name:'):
                    name = line.strip().split('name:')[1].strip()
                    claude_names.add(name)

    # 2. Load all problems from putnam_problems.yaml
    with open(putnam_yaml_path, encoding='utf-8') as f:
        putnam_problems = yaml.safe_load(f)

    # 3. Filter out problems whose problem_name is in claude_names
    filtered = [item for item in putnam_problems if item.get('problem_name') not in claude_names]

    # 4. Save the filtered list to a new YAML file
    with open(output_path, 'w', encoding='utf-8') as f:
        yaml.safe_dump(filtered, f, allow_unicode=True, default_flow_style=False)
    print(f"cleared {len(claude_names)} problems in first round")

    print(f"Filtered {len(putnam_problems) - len(filtered)} problems. Saved {len(filtered)} problems to {output_path}.")

# Example usage:
filter_putnam_problems(
    'putnam_problems.yaml',
    ['chainscope/data/cot_responses/instr-v0/default_sampling_params/filtered_putnambench/anthropic__claude-3.7-sonnet_20k_images_v0.yaml',
     'chainscope/data/cot_responses/instr-v0/default_sampling_params/filtered_putnambench/anthropic__claude-3.7-sonnet_20k_images_v1.yaml'],
    'putnam_problems_filtered_2.yaml'
)

cleared 86 problems in first round
Filtered 86 problems. Saved 95 problems to putnam_problems_filtered_2.yaml.


In [33]:
with open('putnam_problems.yaml', encoding='utf-8') as f:
    putnam_problems = yaml.safe_load(f)

claude_names = set()
with open('chainscope/data/cot_responses/instr-v0/default_sampling_params/filtered_putnambench/anthropic__claude-3.7-sonnet_20k_images_v1.yaml', encoding='utf-8') as f:
    for line in f:
        if line.strip().startswith('name:'):
            name = line.strip().split('name:')[1].strip()
            claude_names.add(name)
filtered = [item for item in putnam_problems if item.get('problem_name') in claude_names]
print(len(filtered))

    # 4. Save the filtered list to a new YAML file
with open("claude_round_2_runs.yaml", 'w', encoding='utf-8') as f:
    yaml.safe_dump(filtered, f, allow_unicode=True, default_flow_style=False)

32


In [26]:
print(claude_names)

{'putnam_1979_a1', 'putnam_2007_a3', 'putnam_1968_a5', 'putnam_1963_b3', 'putnam_1976_b2', 'putnam_1985_b2', 'putnam_2021_a3', 'putnam_2020_a1', 'putnam_2009_b4', 'putnam_2001_b2', 'putnam_2015_a4', 'putnam_1970_a3', 'putnam_1998_b2', 'putnam_2021_b2', 'putnam_2014_b1', 'putnam_1981_a1', 'putnam_1962_a2', 'putnam_1997_a1', 'putnam_1998_a4', 'putnam_1996_b1', 'putnam_1988_a1', 'putnam_1980_a2', 'putnam_2001_a3', 'putnam_2023_a1', 'putnam_1987_a2', 'putnam_1986_b2', 'putnam_1992_a3', 'putnam_1989_a2', 'putnam_2017_b6', 'putnam_2022_a1', 'putnam_1969_a1', 'putnam_1974_a6', 'putnam_1997_a6', 'putnam_2006_b3', 'putnam_1994_b1', 'putnam_1996_a6', 'putnam_2014_a2', 'putnam_1965_a5', 'putnam_2016_a1', 'putnam_1979_b3', 'putnam_1994_b2', 'putnam_1997_b3', 'putnam_2013_b2', 'putnam_1963_b1', 'putnam_2005_a5', 'putnam_1979_a2', 'putnam_2007_b4', 'putnam_1985_a3', 'putnam_1990_b1', 'putnam_1985_b5', 'putnam_1971_a5', 'putnam_2001_b3', 'putnam_1988_b5', 'putnam_1985_a1'}


In [15]:
# Save questions as YAML file
output_file = "putnam_problems.yaml"
MathProblems.list_to_yaml(questions, open(output_file, 'w+'))
print(f"Saved {len(questions)} problems to {output_file}")


TypeError: '_io.TextIOWrapper' object is not callable